In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf

In [4]:
tf.__version__

'2.2.0-rc2'

In [1]:
! mkdir -p /root/.kaggle
! mv kaggle.json /root/.kaggle
import kaggle

In [2]:
! kaggle competitions download -c dogs-vs-cats

100% 543M/543M [00:08<00:00, 73.0MB/s]
100% 543M/543M [00:08<00:00, 69.0MB/s]
 93% 253M/271M [00:05<00:00, 44.7MB/s]
100% 271M/271M [00:06<00:00, 47.3MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 89.4MB/s]


In [0]:
! unzip -q train.zip

In [0]:
# ! ls train

In [0]:
import os 
os.mkdir("data")
os.mkdir("data/cat")
os.mkdir("data/dog")

In [0]:
SRC_FOLDER = "train"
CAT_FODLER = "data/cat"
DOG_FOLDER = "data/dog"

In [9]:
from  shutil import copy
from tqdm import tqdm_notebook
for image in tqdm_notebook(os.listdir(SRC_FOLDER)):
  if image.lower().startswith("cat"):
    copy(os.path.join(SRC_FOLDER, image), os.path.join(CAT_FODLER))
  else:
    copy(os.path.join(SRC_FOLDER, image), os.path.join(DOG_FOLDER))

ERROR! Session/line number was not unique in database. History logging moved to new session 59


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# ! ls data/cat

In [0]:
from tensorflow.keras.applications import VGG16

In [0]:
model = VGG16(include_top=False, input_shape=(224, 224, 3))

In [61]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [62]:
for layer in model.layers:
  layer.trainable = False
for layer in model.layers:
  print(layer.name, layer.trainable)

input_4 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


In [63]:
model.output_shape[1:]

(7, 7, 512)

In [0]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy

In [65]:
from tensorflow.keras import __version__
__version__

'2.3.0-tf'

In [0]:
# top_model = Sequential()
# top_model.add(Flatten(input_shape=(model.output_shape[1:])))
# top_model.add(Dense(256, activation=relu))
# top_model.add(Dropout(0.4))
# top_model.add(Dense(2, activation=softmax))

In [67]:
model.layers[-1].output

<tf.Tensor 'block5_pool_3/Identity:0' shape=(None, 7, 7, 512) dtype=float32>

In [68]:
type(top_model), type(model)

(tensorflow.python.keras.engine.sequential.Sequential,
 tensorflow.python.keras.engine.training.Model)

In [0]:
x = model.layers[-1].output
x = Flatten()(x)
x = Dense(256, relu)(x)
x = Dropout(0.4)(x)
x = Dense(2, softmax)(x)

In [0]:
from tensorflow.keras.models import Model

In [0]:
final_model = Model(inputs=model.input, outputs=x)

In [72]:
final_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [73]:
for layer in final_model.layers:
  print(layer.name, layer.trainable)

input_4 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
flatten_6 True
dense_11 True
dropout_4 True
dense_12 True


In [0]:
final_model.compile("rmsprop", categorical_crossentropy, ["acc"])

In [78]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(rescale=1/255.)
train_gen = idg.flow_from_directory("data", target_size=(224, 224), batch_size=128)

Found 25000 images belonging to 2 classes.


In [0]:
final_model.fit_generator(train_gen, steps_per_epoch=25000//128, epochs=10)

Epoch 1/10
195/195 [==============================] - 103s 527ms/step - loss: 0.2472 - acc: 0.8991
Epoch 2/10
195/195 [==============================] - 101s 520ms/step - loss: 0.2130 - acc: 0.9166
Epoch 3/10
195/195 [==============================] - 101s 518ms/step - loss: 0.1818 - acc: 0.9261
Epoch 4/10
195/195 [==============================] - 101s 518ms/step - loss: 0.1690 - acc: 0.9328
Epoch 5/10
195/195 [==============================] - 101s 515ms/step - loss: 0.1522 - acc: 0.9390
Epoch 6/10
195/195 [==============================] - 101s 515ms/step - loss: 0.1403 - acc: 0.9439
Epoch 7/10
195/195 [==============================] - 101s 519ms/step - loss: 0.1287 - acc: 0.9479
Epoch 8/10
195/195 [==============================] - 101s 516ms/step - loss: 0.1192 - acc: 0.9519
Epoch 9/10
195/195 [==============================] - 101s 518ms/step - loss: 0.1177 - acc: 0.9525
Epoch 10/10
 30/195 [===>..........................] - ETA: 1:21 - loss: 0.0970 - acc: 0.9606